NLP
- NLTK - !pip install nltk
- Datasets - !pip install datasets
- SPACY
- Tensorflow
- Pytorch
- Transformers (HF)

In [1]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to C:\Users\ALIENWARE
[nltk_data]     M15\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to C:\Users\ALIENWARE
[nltk_data]     M15\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [2]:
from datasets import load_dataset

dataset = load_dataset("tweet_eval", "emoji")

c:\Users\ALIENWARE M15\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Extracting data files: 100%|██████████| 6/6 [00:00<00:00, 1004.34it/s]


Dataset tweet_eval downloaded and prepared to C:/Users/ALIENWARE M15/.cache/huggingface/datasets/tweet_eval/emoji/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


100%|██████████| 3/3 [00:00<00:00, 107.22it/s]


In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})

In [4]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [5]:
tr_text = dataset["train"]["text"]
tr_label = dataset["train"]["label"]

val_text = dataset["validation"]["text"]
val_label = dataset["validation"]["label"]

test_text = dataset["test"]["text"]
test_label = dataset["test"]["label"]

In [7]:
import pandas as pd

df = pd.DataFrame()

In [8]:
df['text'] = tr_text + test_text + val_text
df['label'] = tr_label + test_label + val_label
df.head(10)

,text,label
0,Sunday afternoon walking through Venice in the...,12
1,Time for some BBQ and whiskey libations. Chomp...,19
2,Love love love all these people ️ ️ ️ #friends...,0
3,"️ ️ ️ ️ @ Toys""R""Us",0
4,Man these are the funniest kids ever!! That fa...,2
5,"#sandiego @ San Diego, California",11
6,My little ️ ️ ️ ️ ️ #ObsessedWithMyDog @ Cafe ...,0
7,"More #tinyepic things #tinyepicwestern, this o...",19
8,Last night ️ @ Omnia Night Club At Caesars Palace,0
9,friendship at its finest. ....#pixar #toystory...,7


In [9]:
df.shape

(100000, 2)

In [10]:
df['label'].value_counts()

0     21058
1     10252
2      9751
4      6105
7      5293
3      4956
6      4278
18     4056
5      3983
10     3573
17     3063
11     3038
8      3013
9      2737
14     2659
16     2640
12     2640
19     2332
15     2326
13     2247
Name: label, dtype: int64

In [11]:
df2 = df[df['label'].isin([14,17])]
df2.head()

,text,label
31,#Lunch @user best #Ribs in #SanDiego (at @user...,14
35,Surprised everyone by actually surviving a wee...,14
40,Stay classy #SanDiego (@ San Diego Internation...,14
74,@user hi from Tijuana B.C. See you soon,14
77,Birthday cheers with my favorite people!!#26ag...,14


In [12]:
df2['label'].value_counts()

17    3063
14    2659
Name: label, dtype: int64

Preprocessing

In [14]:
sample = df2['text'].iloc[0]
print(sample)

#Lunch @user best #Ribs in #SanDiego (at @user in San Diego, CA)


In [15]:
print(str(sample).replace("@", ""))

#Lunch user best #Ribs in #SanDiego (at user in San Diego, CA)


In [16]:
print(str(sample).replace("#", ""))

Lunch @user best Ribs in SanDiego (at @user in San Diego, CA)


Text Cleaning

In [17]:
df2['text'] = df2['text'].str.replace("@", "") # remove @
df2['text'] = df2['text'].str.replace("#", "") # remove hashtags
df2['text'] = df2['text'].str.replace("\d+", "") # remove numbers
df2['text'] = df2['text'].str.replace("http\S+|www.\S+", "") # remove links
df2['text'] = df2['text'].str.lower() # lower case
df2.head()

C:\Users\ALIENWARE M15\AppData\Local\Temp\ipykernel_9244\706671178.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['text'] = df2['text'].str.replace("@", "") # remove @
C:\Users\ALIENWARE M15\AppData\Local\Temp\ipykernel_9244\706671178.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['text'] = df2['text'].str.replace("#", "") # remove hashtags
C:\Users\ALIENWARE M15\AppData\Local\Temp\ipykernel_9244\706671178.py:3: FutureWarning: The default value of regex will change from True to False in a 

,text,label
31,lunch user best ribs in sandiego (at user in s...,14
35,surprised everyone by actually surviving a wee...,14
40,stay classy sandiego ( san diego international...,14
74,user hi from tijuana b.c. see you soon,14
77,birthday cheers with my favorite people!!again...,14


Feature Extraction

In [18]:
df2.shape

(5722, 2)

In [19]:
# Bag of Words
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()
bow.fit(df2['text']) # creating vocabulary
bow = bow.transform(df2['text']) # creating bag of words

print(bow.shape)

(5722, 11801)


In [32]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

# Tf Idf
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(
    stop_words=stop,
    ngram_range=(1, 1)
)
tfidf.fit(df2['text']) # creating vocabulary
tfidf = tfidf.transform(df2['text']) # creating tfidf
print(tfidf.shape)

(5722, 11668)


Modelling BoW

In [20]:
from sklearn.model_selection import train_test_split as ts
X_train, X_test, y_train, y_test = ts(bow, df2['label'], test_size=0.2, random_state=42)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

from sklearn.metrics import accuracy_score as acc
print(acc(y_test, y_pred))

0.6724890829694323


Modelling TF-IDF

In [30]:
from sklearn.model_selection import train_test_split as ts
X_train, X_test, y_train, y_test = ts(tfidf, df2['label'], test_size=0.2, random_state=42)

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

from sklearn.metrics import accuracy_score as acc
print(acc(y_test, y_pred))

0.4707423580786026


In [33]:
# modelling TfIdf NB
from sklearn.model_selection import train_test_split as ts
X_train, X_test, y_train, y_test = ts(tfidf, df2['label'], test_size=0.2, random_state=42)

from sklearn.naive_bayes import MultinomialNB as NB
nb = NB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)

from sklearn.metrics import accuracy_score as acc
print(acc(y_test, y_pred))

0.8078602620087336
